In [7]:
#importing requried modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
#from google.colab import files
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
%matplotlib inline

In [8]:
dataset = pd.read_csv('SMSSpamCollection.csv',delimiter='	')
dataset.head()

,spam/ham,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
f = feature_extraction.text.CountVectorizer(stop_words ='english')
X = f.fit_transform(dataset["message"])
np.shape(X)

(5572, 8444)

In [11]:
dataset["spam/ham"]= dataset["spam/ham"].map({"spam":1,"ham":0})
X_train,X_test, y_train, y_test = model_selection.train_test_split(X, dataset['spam/ham'], test_size=0.2, random_state=42)


In [12]:
list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.MultinomialNB(alpha=alpha)
    bayes.fit(X_train, y_train)
    score_train[count] = bayes.score(X_train, y_train)
    score_test[count]= bayes.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, bayes.predict(X_test))
    count = count + 1 

In [13]:
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['alpha', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

,alpha,Train Accuracy,Test Accuracy,Test Recall,Test Precision
0,0.00001,0.997308,0.979372,0.919463,0.925676
1,0.11001,0.996635,0.984753,0.966443,0.923077
2,0.22001,0.996410,0.981166,0.966443,0.900000
3,0.33001,0.995961,0.979372,0.966443,0.888889
4,0.44001,0.995513,0.980269,0.966443,0.894410
5,0.55001,0.994840,0.980269,0.959732,0.899371
6,0.66001,0.994840,0.981166,0.959732,0.905063
7,0.77001,0.994840,0.982063,0.959732,0.910828
8,0.88001,0.994615,0.982063,0.959732,0.910828
9,0.99001,0.993942,0.982960,0.959732,0.916667


In [14]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index,:]

alpha             5.720010
Train Accuracy    0.986762
Test Accuracy     0.990135
Test Recall       0.926174
Test Precision    1.000000
Name: 52, dtype: float64